In [1]:
from openai import OpenAI
import pandas as pd
df = pd.read_csv('nouns.csv')
client = OpenAI()

#def p1_translate_(noun):
#    completion = client.chat.completions.create(
#        model="gpt-3.5-turbo",
#messages = [
#            {"role": "system", "content": 
#                "ترجم الاسم المعطي بالدارجة المغربية ل اسم بانجليزية بلا تفسيرات."
#             },
#            {"role": "user", "content": f"{noun}"},
#],
#        temperature=0
#    )
#    return completion.choices[0].message.content
#
#def eng_p1_translate_(noun):
#    completion = client.chat.completions.create(
#        model="gpt-3.5-turbo",
#        messages=[
#            {"role": "system", "content": 
#                "Translate the provided Moroccan Darija dialect noun into an English noun with no explanations."},
#            {"role": "user", "content": f"{noun}"},
#        ],
#        temperature=0
#    )
#    return completion.choices[0].message.content
#
#def p2_translate_(noun):
#    completion = client.chat.completions.create(
#        model="gpt-3.5-turbo",
#messages = [
#            {"role": "system", "content": 
#                "ترجم الاسم المعطية بالدارجة المغربية ل اسم واحد بانجليزية بلا تفسيرات."
#             },
#            {"role": "user", "content": f"{noun}"},
#],
#        temperature=0
#    )
#    return completion.choices[0].message.content
#
#def eng_p2_translate_(noun):
#    completion = client.chat.completions.create(
#        model="gpt-3.5-turbo",
#        messages=[
#            {"role": "system", "content": 
#                "Translate the provided Moroccan Darija dialect noun into one English noun with no explanations."},
#            {"role": "user", "content": f"{noun}"},
#        ],
#        temperature=0
#    )
#    return completion.choices[0].message.content
#
#
#def p3_translate_(noun):
#    completion = client.chat.completions.create(
#        model="gpt-3.5-turbo",
#        messages = [
#            {"role": "system", "content": 
#                "نتا خبير فلهجة الدارجة المغربية وفالترجمة من الدارجة لنجليزية. " + 
#                "ترجم الاسم المعطية بالدارجة المغربية ل اسم واحد بانجليزية بلا تفسيرات."
#             },
#            {"role": "user", "content": f"{noun}"},
#        ],
#        temperature=0
#    )
#    return completion.choices[0].message.content
#def eng_p3_translate_(noun):
#    completion = client.chat.completions.create(
#        model="gpt-3.5-turbo",
#        messages = [
#            {"role": "system", "content": 
#             "You are an expert in Language translation from Moroccan Darija dialect to English. " +
#                "Translate the provided Moroccan Darija dialect noun into one English noun with no explanations."},
#            {"role": "user", "content": f"{noun}"},             
#            ],
#        temperature=0
#    )
#    return completion.choices[0].message.content
#
#def p4_translate_(noun):
#    completion = client.chat.completions.create(
#        model="gpt-3.5-turbo",
#        messages = [
#            {"role": "system", "content": 
#                "نتا خبير فلهجة الدارجة المغربية وفالترجمة من الدارجة لنجليزية. " + 
#                "غادي نعطيك واحد الاسم بالدارجة المغربية، والمهمة ديالك هي تترجمها ل اسم واحد انجليزي بلا تفسيرات."                
#             },
#            {"role": "user", "content": f"{noun}"}
#        ],
#        temperature=0
#    )
#    return completion.choices[0].message.content
#def eng_p4_translate_(noun):
#    completion = client.chat.completions.create(
#        model="gpt-3.5-turbo",
#        messages = [
#            {"role": "system", "content": 
#             "You are an expert in Language translation from Moroccan Darija dialect to English. " +
#             "You will be provided with an noun in Moroccan Darija language, and your task is to translate it into one English noun with no explanations."},
#            {"role": "user", "content": f"{noun}"},
#        ],
#        temperature=0
#    )
#    return completion.choices[0].message.content
#
#
#
def p5_translate_(noun):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": 
                "نتا خبير فلهجة الدارجة المغربية وفالترجمة من الدارجة لنجليزية. " + 
                "غادي نعطيك واحد الاسم بالدارجة المغربية، والمهمة ديالك هي تترجمها ل اسم بانجليزية فكلمة وحدة بلا تفسيرات."                
             },

            {"role": "user", "content": "كاضو"},
            {"role": "assistant", "content": "gift"},
            {"role": "user", "content": "ميكا"},
            {"role": "assistant", "content": "plastic"},
            {"role": "user", "content": "كسوة"},
            {"role": "assistant", "content": "dress"},

            {"role": "user", "content":  f'{noun}' }
        ],
        temperature=0.1  # Deterministic responses
    )
    return completion.choices[0].message.content

def eng_p5_translate_(noun):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": 
             "You are an expert in Language translation from Moroccan Darija dialect to English. " +
             "You will be provided with noun in Moroccan Darija language, and your task is to translate it into an English noun in one word with no explanations."},
            {"role": "user", "content": "كاضو"},
            {"role": "assistant", "content": "gift"},
            {"role": "user", "content": "ميكا"},
            {"role": "assistant", "content": "plastic"},
            {"role": "user", "content": "كسوة"},
            {"role": "assistant", "content": "dress"},


            {"role": "user", "content":  f'{noun}' }
        ],
        temperature=0.1  # Deterministic responses
    )
    return completion.choices[0].message.content




In [2]:

dataset = df.copy()
columns_to_delete = ['n1','n2', 'n3']
dataset.drop(columns=columns_to_delete, inplace=True)

#dataset['p1_transl'] = dataset['darija_ar'].apply(p1_translate_)
#print('1')
#
#dataset['eng_p1_transl'] = dataset['darija_ar'].apply(eng_p1_translate_)
#print('2')
#dataset['p2_transl'] = dataset['darija_ar'].apply(p2_translate_)
#print('3')
#dataset['eng_p2_transl'] = dataset['darija_ar'].apply(eng_p2_translate_)
#print('4')
#dataset['p3_transl'] = dataset['darija_ar'].apply(p3_translate_)
#print('5')
#dataset['eng_p3_transl'] = dataset['darija_ar'].apply(eng_p3_translate_)
#print('6')
#dataset['p4_transl'] = dataset['darija_ar'].apply(p4_translate_)
#print('71')
#dataset['eng_p4_transl'] = dataset['darija_ar'].apply(eng_p4_translate_)
#print('8')
dataset['p5_transl'] = dataset['darija_ar'].apply(p5_translate_)
print('9')
dataset['eng_p5_transl'] = dataset['darija_ar'].apply(eng_p5_translate_)
print('1')
dataset.to_csv('noun1.csv', index=False)

9
1


In [1]:
import matplotlib.pyplot as plt
from nltk.corpus import wordnet as wn
import pandas as pd
dataset = pd.read_csv('noun1.csv')


eng = dataset['eng']
#p1_transl = dataset['p1_transl']
#eng_p1_transl = dataset['eng_p1_transl']
#p2_transl = dataset['p2_transl']
#eng_p2_transl = dataset['eng_p2_transl']
#p3_transl = dataset['p3_transl']
#eng_p3_transl = dataset['eng_p3_transl']
#p4_transl = dataset['p4_transl']
#eng_p4_transl = dataset['eng_p4_transl']
p5_transl = dataset['p5_transl']
eng_p5_transl = dataset['eng_p5_transl']


def max_path_similarity(word1, word2):
    """Calculate the maximum path similarity between two words."""

    synsets1 = wn.synsets(word1, pos=wn.VERB)
    synsets2 = wn.synsets(word2, pos=wn.VERB)
    max_score = 0
    for syn1 in synsets1:
        for syn2 in synsets2:
            score = syn1.path_similarity(syn2)
            if score and score > max_score:
                max_score = score
    return max_score

def evaluate_pairs(transl,threshold=0.5):
    """Evaluating each pair using WordNet path similarity and a threshold."""
    results = []
    for i in range(len(transl)):
        word1, word2 = transl[i], eng[i]
        similarity = max_path_similarity(word1, word2)
        results.append((word1, word2, similarity, similarity >= threshold))
    return results

#results_p1 = evaluate_pairs(p1_transl)
#results_eng_p1 = evaluate_pairs(eng_p1_transl)
#results_p2 = evaluate_pairs(p2_transl)
#results_eng_p2 = evaluate_pairs(eng_p2_transl)
#results_p3 = evaluate_pairs(p3_transl)
#results_eng_p3 = evaluate_pairs(eng_p3_transl)
#results_p4 = evaluate_pairs(p4_transl)
#results_eng_p4 = evaluate_pairs(eng_p4_transl)
results_p5 = evaluate_pairs(p5_transl)
results_eng_p5 = evaluate_pairs(eng_p5_transl)

#dataset['similariy_p1'] = [item[2] for item in results_p1]
#dataset['similarity_eng_p1'] = [item[2] for item in results_eng_p1]
#dataset['similarity_p2'] = [item[2] for item in results_p2]
#dataset['similarity_eng_p2'] = [item[2] for item in results_eng_p2]
#dataset['similarity_p3'] = [item[2] for item in results_p3]
#dataset['similarity_eng_p3'] = [item[2] for item in results_eng_p3]
#dataset['similarity_p4'] = [item[2] for item in results_p4]
#dataset['similarity_eng_p4'] = [item[2] for item in results_eng_p4]
dataset['similarity_p5'] = [item[2] for item in results_p5]
dataset['similarity_eng_p5'] = [item[2] for item in results_eng_p5]

#p1_s = dataset['similariy_p1']
#ep1_s = dataset['similarity_eng_p1']
#p2_s = dataset['similarity_p2'] 
#ep2_s = dataset['similarity_eng_p2'] 
#p3_s = dataset['similarity_p3'] 
#ep3_s = dataset['similarity_eng_p3'] 
#p4_s = dataset['similarity_p4'] 
#ep4_s = dataset['similarity_eng_p4']
p5_s = dataset['similarity_p5'] 
ep5_s = dataset['similarity_eng_p5']


#p1= dataset['similariy_p1'].mean()
#ep1= dataset['similarity_eng_p1'].mean()
#p2= dataset['similarity_p2'].mean()
#ep2= dataset['similarity_eng_p2'].mean()
#p3= dataset['similarity_p3'].mean()
#ep3= dataset['similarity_eng_p3'].mean()
#p4= dataset['similarity_p4'].mean()
#ep4= dataset['similarity_eng_p4'].mean()
p5= dataset['similarity_p5'].mean()
ep5= dataset['similarity_eng_p5'].mean()

dataset.describe()

,similarity_p5,similarity_eng_p5
count,1328.000000,1328.000000
mean,0.268259,0.260317
std,0.420831,0.415619
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.500000,0.333333
max,1.000000,1.000000


In [4]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Box(y=dataset['similarity_p5'], name='Darija fifth prompt'))
fig.add_trace(go.Box(y=dataset['similarity_eng_p5'], name='English fifth prompt'))

# Update layout
fig.update_layout(
    title='Box Plot of Similarity Scores for all the prompts.',
    yaxis_title='Similarity Score',
    xaxis_title='Prompt',
    boxmode='group'
)

# Show plot
fig.show()

In [4]:
import plotly.graph_objects as go


fig = go.Figure()

fig.add_trace(go.Histogram(histfunc="count", y=dataset['similarity_p5'], x=dataset['similarity_p5'], name="Darija prompt"))
fig.add_trace(go.Histogram(histfunc="count", y=dataset['similarity_eng_p5'], x=dataset['similarity_eng_p5'], name="English prompt"))

fig.show()

In [3]:
result = dataset[dataset['similarity_p5'] == 0].values.tolist()
print(result)


[['قهوة', 'coffee', 'coffee', 'coffee', 0.0, 0.0], ['قرعة', 'bottle', 'lottery', 'lottery', 0.0, 0.0], ['طوموبيل', 'car', 'car', 'car', 0.0, 0.0], ['موتور', 'motorcycle', 'car', 'engine', 0.0, 0.0], ['شفار', 'thief', 'thief', 'thief', 0.0, 0.0], ['مولحانوت', 'grocery seller', 'Supermarket', 'Supermarket', 0.0, 0.0], ['مولبيسري', 'grocery seller', 'messenger', 'hairdresser', 0.0, 0.0], ['بارتمة', 'apartment', 'backpack', 'carpet', 0.0, 0.0], ['نعاس', 'sleep', 'sleepiness', 'drowsiness', 0.0, 0.0], ['ولد', 'boy', 'boy', 'boy', 0.0, 0.0], ['بنت', 'girl', 'girl', 'girl', 0.0, 0.0], ['شارع', 'avenue', 'street', 'street', 0.0, 0.0], ['درب', 'alley', 'path', 'path', 0.0, 0.0], ['حومة', 'neighborhood', 'neighborhood', 'meat', 0.0, 0.0], ['دروج', 'stairs', 'shoes', 'lessons', 0.0, 0.0], ['نخلة', 'date palm', 'palm', 'palm', 0.0, 0.0], ['لغة', 'language', 'language', 'language', 0.0, 0.0], ['عام', 'year', 'year', 'year', 0.0, 0.0], ['فخور', 'proud', 'proud', 'proud', 0.0, 0.0], ['ليوم', 'today',

In [1]:
from nltk.corpus import wordnet as wn

def max_path_similarity(word1, word2):
    """Calculate the maximum path similarity between two words."""

    synsets1 = wn.synsets(word1, pos=wn.VERB)
    synsets2 = wn.synsets(word2, pos=wn.VERB)
    max_score = 0
    for syn1 in synsets1:
        for syn2 in synsets2:
            score = syn1.path_similarity(syn2)
            if score and score > max_score:
                max_score = score
    return max_score

word1 = 'advocated'
synsets1 = wn.synsets(word1, pos=wn.VERB)
print(synsets1)

[Synset('recommend.v.01'), Synset('preach.v.02')]
